In [1]:
import json
import dask
import dask_awkward as dak
import awkward as ak
import numpy as np
from coffea import dataset_tools
from coffea.nanoevents import NanoEventsFactory, PFNanoAODSchema
import fastjet
import time
import os
import warnings
import scipy

In [2]:
warnings.filterwarnings("ignore", module="coffea.*")
warnings.filterwarnings("ignore", "Found duplicate branch")
warnings.filterwarnings("ignore", "Missing cross-reference index for")
warnings.filterwarnings("ignore", "dcut")
warnings.filterwarnings("ignore", "Please ensure")
warnings.filterwarnings("ignore", "invalid value")

In [9]:
path = '/cms/cephfs/data/store/user/cmoore24/samples/hgg/'
files = os.listdir(path)

In [47]:
events = NanoEventsFactory.from_root(
    {f'{path}{files[2]}':"/Events"},
    delayed=False,
    schemaclass=PFNanoAODSchema,
    metadata={'dataset':'hgg'}
).events()

In [48]:
fatjetSelect = (
            (events.FatJet.pt >= 450)
            & (events.FatJet.pt <= 1200)
            & (abs(events.FatJet.eta) <= 2.4)
            & (events.FatJet.msoftdrop >= 40)
            & (events.FatJet.msoftdrop <= 200)
        )

In [49]:
cut_to_fix_softdrop = (ak.num(events.FatJet.constituents.pf, axis=2) > 0)
events = events[ak.all(cut_to_fix_softdrop, axis=1)]

In [50]:
events["goodjets"] = events.FatJet[fatjetSelect]
mask = ~ak.is_none(ak.firsts(events.goodjets))
events = events[mask]

In [51]:
events['goodjets'] = events.goodjets[(ak.local_index(events.goodjets, axis=1) == 0)]

In [52]:
jetdef = fastjet.JetDefinition(
            fastjet.cambridge_algorithm, 1.0
        )

In [53]:
pf = ak.flatten(events.goodjets.constituents.pf, axis=1)

In [55]:
cluster = fastjet.ClusterSequence(pf, jetdef)

In [112]:
cluster.exclusive_jets_lund_declusterings(1)[0]

<Array [[{Delta: 0.546, kt: 27.5}, ..., {...}]] type='1 * var * {Delta: flo...'>

In [111]:
cluster.get_child()

['_jetdef',
 '_jagedness',
 '_flag',
 '_internalrep',
 '__module__',
 '__init__',
 '_check_jaggedness',
 '_check_listoffset_index',
 '_check_record',
 '_check_indexed',
 '_check_listoffset_subtree',
 '_check_general',
 '_check_general_jaggedness',
 '_check_listoffset',
 'jet_def',
 'inclusive_jets',
 'unclustered_particles',
 'exclusive_jets',
 'exclusive_jets_up_to',
 'exclusive_jets_ycut',
 'constituent_index',
 'constituents',
 'exclusive_jets_constituent_index',
 'exclusive_jets_constituents',
 'exclusive_jets_softdrop_grooming',
 'exclusive_jets_energy_correlator',
 'exclusive_jets_lund_declusterings',
 'exclusive_dmerge',
 'exclusive_dmerge_max',
 'exclusive_ymerge_max',
 'exclusive_ymerge',
 'Q',
 'Q2',
 'exclusive_subjets',
 'exclusive_subjets_up_to',
 'exclusive_subdmerge',
 'exclusive_subdmerge_max',
 'n_exclusive_subjets',
 'has_parents',
 'has_child',
 'jet_scale_for_algorithm',
 'unique_history_order',
 'n_particles',
 'n_exclusive_jets',
 'childless_pseudojets',
 'jets',
